# Toycode for PyTorch-Lightning

So far I have bumped into many technical difficulties related to Pytorch-lightning and Tensorboard. In this notebook, I will use a simple model and dataset to experiment with different functionalities including: 

<ul>
    <li>Tensorboard logging</li>
    <li>Callback</li> 
    <li>Freeze parameters</li>
</ul>

Other things I need to experiment 

<ul>
    <li>Rouge metric and other metrics</li>
    <li>Optimizer and scheduler</li>
</ul>
    

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from transformers import AdamW

In [12]:
# Generate 1000 points
X = (np.random.rand(1000, 5) ** 2 * 5).tolist()
# Labels are the norms of 1000 points
y = (np.sqrt(np.sum(np.square(X), axis = 1))).tolist()

X_train, y_train = X[:800], y[:800]
X_val, y_val = X[800:], y[800:]

In [3]:
class MyDataset(Dataset): 
    def __init__(self, X, y): 
        super().__init__()
        self.X = X
        self.y = y
        
    def __len__(self): 
        return len(y)
    
    def __getitem__(self,idx):
        return {
            'source': torch.tensor(X[idx]), 
            'target': torch.tensor(y[idx])
        }

In [9]:
class MyModel(pl.LightningModule): 
    ''' Part 1: Define the architecture of model in init '''
    def __init__(self, hparams):
        super(MyModel, self).__init__()
        self.layer1 = nn.Linear(5, 10)
        self.layer2 = nn.Linear(10, 8)
        self.layer3 = nn.Linear(8, 1)
        self.hparams = hparams 
        
    ''' Part 2: Define the forward propagation '''
    def forward(self, x): 
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x
    
    ''' Part 3: Prepare optimizer and scheduler '''
    def configure_optimizer(self): 
        optimizer = AdamW(self.parameters, lr = self.hparams['learning_rate'])
        return optimizer
    
    ''' Part 4: Training logic '''
    def training_step(self, batch, batch_idx): 
        X = batch['source']
        y = batch['target']
        y_hat = self(X)    # Calls forward function 
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss)
        return loss
    
    ''' Part 5: Validation logic '''
    def validation_step(self, batch, batch_idx): 
        X = batch['source']
        y = batch['target']
        y_hat = self(X)
        loss = F.mse_loss(y_hat, y)
        self.log('val_loss', loss)
        
    ''' Part 6: Data loaders '''
    def train_dataloader(self): 
        dataset = MyDataset(X[:800], y[:800])
        return DataLoader(dataset, batch_size = hparams['batch_size'])
    
    def train_dataloader(self): 
        dataset = MyDataset(X[:800], y[:800])
        return DataLoader(dataset, batch_size = hparams['batch_size'])
    
    def train_dataloader(self): 
        dataset = MyDataset(X[:800], y[:800])
        return DataLoader(dataset, batch_size = hparams['batch_size'])

In [10]:
hparams = {
    'learning_rate': 3e-4, 
    'batch_size': 16
}

In [11]:


model = MyModel(hparams)
trainer = pl.Trainer
trainer.fit(model,  )